In [59]:
import json
import os
import urllib
import ssl

def download_and_load_file(file_path, url):
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE

    if not os.path.exists(file_path):
        with urllib.request.urlopen(url, context=ssl_context) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
    else:
        with open(file_path, "r", encoding="utf-8") as file:
            text_data = file.read()

    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    return data


file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/refs/heads"
    "/main/alpaca_data.json"
)

data = download_and_load_file(file_path, url)
print("Number of entries:", len(data))


Number of entries: 52002


In [2]:
data[0]

{'instruction': 'Evaluate the following phrase by transforming it into the spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".'}

In [3]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

In [4]:
print(format_input(data[940]))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the sentence.

### Input:
The lecture was delivered in a clear manner.


In [5]:
response = format_input(data[940]) + (f"\n\n### Response\n{data[940]['output']}")

In [6]:
print(response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Rewrite the sentence.

### Input:
The lecture was delivered in a clear manner.

### Response
The lecture was delivered clearly.


In [68]:
# train_data = int(len(data)*0.7)
# test_data = int(len(data)*0.2)
# validate_data = int(len(data)*0.1)
train_data = data[:int(len(data)*0.5)]
test_data = data[int(len(data)*0.5):int(len(data)*0.05)+int(len(data)*0.5)]
validate_data = data[int(len(data)*0.95):]
print(len(train_data))

26001


In [66]:
print(len(test_data))


2600


In [69]:
print(len(validate_data))

2601


In [10]:
from torch.utils.data import Dataset

In [11]:
import torch,tiktoken

In [12]:
class Instruction_Dataset(Dataset):
    def __init__(self,data, tokenizer):
        super().__init__()
        self.data = data
        self.encoded = []

        for entry in data:
            instruction_input = format(entry)
            response = f"\n\n### Response:\n{entry['output']}"
            full_entry = instruction_input + response
            self.encoded.append(tokenizer.encode(full_entry))

    def __getitem__(self, index) :
        return self.encoded[index]
    
    def __len__(self):
        return len(self.encoded)


In [13]:
# pip install torch

In [14]:
# now what we will do is we want in each batch every input and target sequence to have same length we ensure that by first adding 1 more 50256 token id to each sequences (sequence having less no of token than the sequence having highest no of token will also have max_len - len +1 later while returning on input we will exclude last token in each sequence) 
# now to get target sequence coressponding to each input sequence we shidt input sequence by 1 towards left and last token will be the 50256
# Also for the target tensor or sequence we replace all the end of text token with -100 except the 1st one in the sequence which allows cross entropy loss function to ignore these tokens while calculating loss later
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
    allowed_max_length=None,
    device="cpu"
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # New: Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index

        # New: Optionally truncate to maximum sequence length
        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]

        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor

In [15]:
# lets test with dummy input batches
inputs_1 = [0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [7, 8, 9]

batch = (
    inputs_1,
    inputs_2,
    inputs_3
)

inputs, targets = custom_collate_fn(batch)
print(inputs)
print(targets)

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256,  -100,  -100,  -100],
        [    8,     9, 50256,  -100,  -100]])


In [16]:
# now it is time to work on data loaders which allows to have multiple batches by arranging the input sequences in a batch of batch size 

from torch.utils.data import DataLoader



In [17]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.enabled)

2.5.1+cu121
12.1
True


In [18]:
# !pip uninstall torch torchvision torchaudio -y

In [19]:
# !pyenv local 3.11.9


In [20]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [21]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [22]:
from functools import partial
customized_collate_fn = partial(custom_collate_fn, device=device, allowed_max_length=1024)#it will auto give the custom_collate_fn these parameters 

In [23]:
import tiktoken

In [70]:
from torch.utils.data import DataLoader


num_workers = 0
batch_size = 8

torch.manual_seed(123)
tokenizer = tiktoken.get_encoding("gpt2")

train_dataset = Instruction_Dataset(train_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

val_dataset = Instruction_Dataset(validate_data, tokenizer)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

test_dataset = Instruction_Dataset(test_data, tokenizer)
test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    collate_fn=customized_collate_fn,#after arranging the sequences in batch of batchsize no of sequences the batch is passed to collate_fn which returns the batch with target sequences also 
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

In [71]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

Train loader:
torch.Size([8, 261]) torch.Size([8, 261])
torch.Size([8, 271]) torch.Size([8, 271])
torch.Size([8, 305]) torch.Size([8, 305])
torch.Size([8, 385]) torch.Size([8, 385])
torch.Size([8, 1024]) torch.Size([8, 1024])
torch.Size([8, 352]) torch.Size([8, 352])
torch.Size([8, 594]) torch.Size([8, 594])
torch.Size([8, 252]) torch.Size([8, 252])
torch.Size([8, 238]) torch.Size([8, 238])
torch.Size([8, 313]) torch.Size([8, 313])
torch.Size([8, 312]) torch.Size([8, 312])
torch.Size([8, 209]) torch.Size([8, 209])
torch.Size([8, 372]) torch.Size([8, 372])
torch.Size([8, 378]) torch.Size([8, 378])
torch.Size([8, 470]) torch.Size([8, 470])
torch.Size([8, 331]) torch.Size([8, 331])
torch.Size([8, 298]) torch.Size([8, 298])
torch.Size([8, 298]) torch.Size([8, 298])
torch.Size([8, 219]) torch.Size([8, 219])
torch.Size([8, 180]) torch.Size([8, 180])
torch.Size([8, 248]) torch.Size([8, 248])
torch.Size([8, 270]) torch.Size([8, 270])
torch.Size([8, 251]) torch.Size([8, 251])
torch.Size([8, 209

In [26]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [27]:
#Now it is time to load pre-trainned llm model
from Fundamentals.gpt_download import download_and_load_gpt2
from Foundational.utils.model import GPT_Model
from Foundational.utils.loadwt_into_gpt import load_weights_into_gpt

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(
    model_size=model_size,
    models_dir="gpt2"
)

model = GPT_Model(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();

c:\Users\ASUS\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
c:\Users\ASUS\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\355M\checkpoint


c:\Users\ASUS\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\355M\encoder.json


c:\Users\ASUS\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

File already exists and is up-to-date: gpt2\355M\model.ckpt.index


c:\Users\ASUS\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\355M\model.ckpt.meta


c:\Users\ASUS\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up-to-date: gpt2\355M\vocab.bpe


In [28]:
from Foundational.utils.generate import generate


In [29]:
import torch
torch.manual_seed(123)
input_text = format_input(validate_data[1])
print(input_text)
print(validate_data[0]['output'])

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Reword the following sentence to the future tense.

### Input:
He is reading a novel inspired by his grandmother.
The primary function of the human heart is to pump blood throughout the body, delivering oxygen and nutrients to tissues and removing carbon dioxide and other wastes.


In [30]:
# import tiktoken

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    print(flat)
    return tokenizer.decode(flat.tolist())

# start_context = "Every effort moves you"
# tokenizer = tiktoken.get_encoding("gpt2")

# token_ids = generate_text_simple(
#     model=model,
#     idx=text_to_token_ids(start_context, tokenizer),
#     max_new_tokens=10,
#     context_size=GPT_CONFIG_124M["context_length"]
# )

# print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

In [31]:
token_ids = generate(
    model=model,
    idx=text_to_token_ids(input_text, tokenizer),
    max_new_tokens=35,
    context_size=BASE_CONFIG["context_length"],
    eos_id=50256,
)
generated_text = token_ids_to_text(token_ids, tokenizer)

tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198, 30003,   585,   262,  1708,  6827,   284,
          262,  2003, 20170,    13,   198,   198, 21017, 23412,    25,   198,
         1544,   318,  3555,   257,  5337,  7867,   416,   465, 18410,    13,
          198,   198, 21017, 25235,    25,   198,   198,  1544,   318,  3555,
          257,  5337,  7867,   416,   465, 18410,    13,   198,   198, 21017,
        46486,    25,   198,   198, 16594,   257,  2882,   326, 20431, 32543,
          262,  2581,    13,   198,   198])


In [32]:
response_text = generated_text[len(input_text):].strip()
print(response_text)

### Output:

He is reading a novel inspired by his grandmother.

### Instruction:

Write a response that appropriately completes the request.


In [33]:
# here we see the ouput is not good and accurate that why we cant depend on pretrainned model 
# now we will train the model with our own data to make the accuracy high 
#lets check loss of this pretrainned model
from Foundational.utils.calc_loader_loss import calc_loader_loss
from Foundational.utils.calc_loader_loss import calc_batch_loss

In [34]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loader_loss(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loader_loss(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
with torch.no_grad():
    train_loss = calc_loader_loss(train_loader, device=device, model=model)
    val_loss = calc_loader_loss(val_loader,device=device,model=model)

print(train_loss,val_loss)

tensor(2.8279, device='cuda:0') tensor(2.8338, device='cuda:0')


In [36]:
def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

In [37]:
# pre trainning the llm model 
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_batch_loss(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel() # Returns the total number of elements (or tokens) in the input_batch.
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0: 
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

        # torch.save({
        #     "model_state_dict": model.state_dict(),
        #     "optimizer_state_dict": optimizer.state_dict(),
        #     }, 
        #     "model_and_optimizer.pth"
        #         )

    return train_losses, val_losses, track_tokens_seen

In [38]:
print(device)

cuda


In [ ]:
import time

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.00005, weight_decay=0.1)

num_epochs = 1

checkpoint = torch.load('checkpoint_epoch1.pth', map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch'] + 1
train_loss = checkpoint.get('train_loss', None)

train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context=format_input(validate_data[0]), tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_25460\2008901580.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint_epoch1.pth', map_locatio

Ep 1 (Step 000000): Train loss 0.483, Val loss 0.707
Ep 1 (Step 000005): Train loss 0.436, Val loss 0.709
Ep 1 (Step 000010): Train loss 0.344, Val loss 0.716
Ep 1 (Step 000015): Train loss 0.364, Val loss 0.716
Ep 1 (Step 000020): Train loss 0.430, Val loss 0.722
Ep 1 (Step 000025): Train loss 0.396, Val loss 0.715
Ep 1 (Step 000030): Train loss 0.424, Val loss 0.715
Ep 1 (Step 000035): Train loss 0.407, Val loss 0.711
Ep 1 (Step 000040): Train loss 0.347, Val loss 0.705
Ep 1 (Step 000045): Train loss 0.392, Val loss 0.702
Ep 1 (Step 000050): Train loss 0.353, Val loss 0.710
Ep 1 (Step 000055): Train loss 0.368, Val loss 0.729
Ep 1 (Step 000060): Train loss 0.349, Val loss 0.729
Ep 1 (Step 000065): Train loss 0.339, Val loss 0.729
Ep 1 (Step 000070): Train loss 0.280, Val loss 0.734
Ep 1 (Step 000075): Train loss 0.321, Val loss 0.714
Ep 1 (Step 000080): Train loss 0.278, Val loss 0.712
Ep 1 (Step 000085): Train loss 0.322, Val loss 0.716
Ep 1 (Step 000090): Train loss 0.306, Val loss

In [40]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(figsize=(5, 3))

    # Plot training and validation loss against epochs
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))  # only show integer labels on x-axis

    # Create a second x-axis for tokens seen
    ax2 = ax1.twiny()  # Create a second x-axis that shares the same y-axis
    ax2.plot(tokens_seen, train_losses, alpha=0)  # Invisible plot for aligning ticks
    ax2.set_xlabel("Tokens seen")

    fig.tight_layout()  # Adjust layout to make room
    plt.savefig("loss-plot.pdf")
    plt.show()

In [41]:
# Save model and optimizer states
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': 2,   # your current epoch
    'train_loss': train_loss
}, 'checkpoint.pth')

In [42]:
# checkpoint = torch.load('checkpoint_epoch1.pth', map_location=device)

# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# start_epoch = checkpoint['epoch'] + 1
# train_loss = checkpoint.get('train_loss', None)

# print(train_loss)
# now this saves the parameters in the model and optimizer and later while doing finetunning we can resume from this saved state 

In [43]:
# num_epochs = 3

# train_losses, val_losses, tokens_seen = train_model_simple(
#     model, train_loader, val_loader, optimizer, device,
#     num_epochs=num_epochs, eval_freq=5, eval_iter=5,
#     start_context=format_input(validate_data[0]), tokenizer=tokenizer
# )

In [44]:
torch.manual_seed(123)


for entry in test_data[:3]:

    input_text = format_input(entry)

    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = (
        generated_text[len(input_text):]
        .replace("### Response:", "")
        .strip()
)

    print(input_text)
    print(f"\nCorrect response:\n>> {entry['output']}")
    print(f"\nModel response:\n>> {response_text.strip()}")
    print("-------------------------------------")

tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198,  2061,   318,   262, 28873, 47625,   329,
          705,    33, 19335,   286,  5800, 30960,   198, 21017, 18261,    25,
          198,   464, 28873, 47625,   329,   705,    33, 19335,   286,  5800,
            6,   318,   347,    13,  3351,    13], device='cuda:0')
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is the abbreviation for 'Bachelor of Science'?

Correct response:
>> The abbreviation for 'Bachelor of Science' is B.Sc.

Model response:
>> The abbreviation for 'Bachelor of Science' is B.Sc.
-------------------------------------
tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    2

### Evaluating the Model


<div class="alert alert-block alert-warning">

Most importantly, we can see that model evaluation is not as straightforward as in the
previous chapter, where we simply calculated the percentage of correct spam/non-spam
class labels to obtain the classification accuracy. 

In practice, instruction-finetuned LLMs
such as chatbots are evaluated via multiple approaches:

1. Short-answer and multiple choice benchmarks such as MMLU ("Measuring
Massive Multitask Language Understanding," https://arxiv.org/abs/2009.
03300), which test the general knowledge of a model.

2. Human preference comparison to other LLMs, such as LMSYS chatbot
arena (https://arena.lmsys.org).

3. Automated conversational benchmarks, where another LLM like GPT-4 is
used to evaluate the responses, such as AlpacaEval (https://tatsulab.github.io/alpaca_eval/).
completes the request.

</div>

In [45]:
from tqdm import tqdm

for i, entry in tqdm(enumerate(test_data), total=len(test_data)):

    input_text = format_input(entry)

    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)
    response_text = generated_text[len(input_text):].replace("### Response:", "").strip()

    test_data[i]["model_response"] = response_text


with open("instruction-data-with-response.json", "w") as file:
    json.dump(test_data, file, indent=4)  # "indent" for pretty-printing

  0%|          | 1/220 [00:01<06:00,  1.65s/it]

tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198,  2061,   318,   262, 28873, 47625,   329,
          705,    33, 19335,   286,  5800, 30960,   198, 21017, 18261,    25,
          198,   464, 28873, 47625,   329,   705,    33, 19335,   286,  5800,
            6,   318,   347,    13,  3351,    13], device='cuda:0')


  1%|          | 2/220 [00:02<04:28,  1.23s/it]

tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198,  3103,  1851,  1542, 48829,   284, 10700,
           13,   198, 21017, 18261,    25,   198,  1270, 48829,   318, 20343,
        10700,    13], device='cuda:0')


  1%|▏         | 3/220 [00:03<04:39,  1.29s/it]

...

tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198,  9487,  1958,   262,  1708,  3709,    25,
        17026,    11,  8278,    11, 26241,    13,   198, 20560,    25,   198,
           33, 35298,   198, 31087,   198,    51,  8254,   198, 21017, 18261,
           25,   198,    33, 35298,   198, 31087,   198,    51,  8254],
       device='cuda:0')


100%|█████████▉| 219/220 [46:33<00:43, 43.99s/it]

tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198,  7469,   500,   262,  3381,   705,   343,
         1647,  4458,   198, 21017, 18261,    25,   198,     1,    40,   892,
          340,   338, 25304,   326,   314,  1101,   287,   257,  2776,   351,
          257,   582,   508,   318,   845,  9112,   290,  1464,  5419,   502,
          503,    13,  2102,    11,   339,   318,   845, 11334,   290,  1464,
         3382,   284,   760,   644,   314,   892,   546,   465,   790,  2551,
           13,   314,   892,   340,   338, 25304,   326,   314,  1101,   287,
          257,  2776,   351,   257,   582,   508,   318,   845,  9112,   290,
         1464,  5419,   502,   503,    13,  2102,    11,   339,   318,   845,
        11334,   290,  1464,  3382,   284,   760,   644,   314,   892,   546,
          465,   790,  2551,    13,   314,   892,   340,   338, 

100%|██████████| 220/220 [46:35<00:00, 12.71s/it]

tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198,  8291, 17660,   705, 14618,     6,   656,
         2679,    13,   198, 21017, 18261,    25,   198,    54,   494,   410,
         8207,   479,   455,   316,  1658,   479,   455,   316,  1658,    30],
       device='cuda:0')


In [46]:
import urllib.request

def query_model(
    prompt,
    model="llama3",
    url="http://localhost:11434/api/chat"
):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }


    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

In [ ]:
model = "llama3"
# result = query_model("What do Llamas eat?", model)
# print(result)

HTTPError: HTTP Error 500: Internal Server Error

In [48]:
# for entry in test_data[:3]:
#     prompt = (
#         f"Given the input `{format_input(entry)}` "
#         f"and correct output `{entry['output']}`, "
#         f"score the model response `{entry['model_response']}`"
#         f" on a scale from 0 to 100, where 100 is the best score. "
#     )
#     print("\nDataset response:")
#     print(">>", entry['output'])
#     print("\nModel response:")
#     print(">>", entry["model_response"])
#     print("\nScore:")
#     print(">>", query_model(prompt))
#     print("\n-------------------------")

In [53]:
def generate_model_scores(json_data, json_key, model="llama3"):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"Respond with the integer number only."
        )
        score = query_model(prompt, model)
        try:
            scores.append(int(score))
        except ValueError:
            print(f"Could not convert score: {score}")
            continue

    return scores

In [54]:
generate_model_scores(test_data,'model_response')

Scoring entries:   0%|          | 0/220 [00:00<?, ?it/s]

Scoring entries:  28%|██▊       | 61/220 [02:49<07:42,  2.91s/it]

Could not convert score: **Subject: 20**

How does a virus infect a cell?


Scoring entries:  71%|███████   | 156/220 [07:01<04:54,  4.61s/it]

Could not convert score: I'd be happy to help!

The model's response for "Tomato" was incorrect, so I'll give it a score of 0.

The model's response for "Vegetable" was also incorrect, so I'll give it a score of 0 again.

Here are the scores:

* Tomato: 0
* Vegetable: 0

So, the average score is (0 + 0) / 2 = 0.


Scoring entries: 100%|██████████| 220/220 [09:48<00:00,  2.67s/it]


[95,
 4,
 4,
 20,
 100,
 20,
 60,
 80,
 20,
 100,
 44,
 20,
 20,
 20,
 60,
 60,
 20,
 20,
 4,
 98,
 20,
 20,
 4,
 85,
 20,
 80,
 60,
 4,
 20,
 95,
 85,
 6,
 60,
 85,
 100,
 20,
 60,
 60,
 4,
 20,
 85,
 20,
 2,
 4,
 20,
 20,
 80,
 60,
 20,
 20,
 20,
 60,
 20,
 20,
 80,
 20,
 20,
 20,
 20,
 20,
 20,
 0,
 60,
 20,
 20,
 95,
 95,
 20,
 85,
 98,
 20,
 20,
 60,
 80,
 67,
 20,
 40,
 85,
 60,
 4,
 20,
 80,
 20,
 20,
 20,
 20,
 100,
 20,
 4,
 100,
 20,
 20,
 60,
 20,
 4,
 100,
 94,
 67,
 20,
 80,
 60,
 4,
 4,
 20,
 20,
 95,
 20,
 12,
 80,
 20,
 98,
 80,
 20,
 20,
 0,
 80,
 98,
 12,
 20,
 20,
 80,
 20,
 20,
 20,
 0,
 20,
 60,
 80,
 20,
 20,
 96,
 20,
 20,
 95,
 20,
 20,
 4,
 95,
 80,
 40,
 20,
 20,
 98,
 20,
 4,
 100,
 50,
 20,
 20,
 80,
 60,
 98,
 20,
 4,
 20,
 100,
 20,
 80,
 80,
 40,
 20,
 60,
 0,
 20,
 20,
 4,
 4,
 20,
 0,
 4,
 80,
 20,
 95,
 95,
 100,
 20,
 95,
 4,
 80,
 95,
 80,
 80,
 20,
 80,
 20,
 20,
 80,
 0,
 60,
 85,
 20,
 98,
 43,
 98,
 80,
 4,
 95,
 98,
 20,
 100,
 80,
 40,
 4,
 85,

In [ ]:
torch.cuda.empty_cache()

In [55]:
scores = [95,
 4,
 4,
 20,
 100,
 20,
 60,
 80,
 20,
 100,
 44,
 20,
 20,
 20,
 60,
 60,
 20,
 20,
 4,
 98,
 20,
 20,
 4,
 85,
 20,
 80,
 60,
 4,
 20,
 95,
 85,
 6,
 60,
 85,
 100,
 20,
 60,
 60,
 4,
 20,
 85,
 20,
 2,
 4,
 20,
 20,
 80,
 60,
 20,
 20,
 20,
 60,
 20,
 20,
 80,
 20,
 20,
 20,
 20,
 20,
 20,
 0,
 60,
 20,
 20,
 95,
 95,
 20,
 85,
 98,
 20,
 20,
 60,
 80,
 67,
 20,
 40,
 85,
 60,
 4,
 20,
 80,
 20,
 20,
 20,
 20,
 100,
 20,
 4,
 100,
 20,
 20,
 60,
 20,
 4,
 100,
 94,
 67,
 20,
 80,
 60,
 4,
 4,
 20,
 20,
 95,
 20,
 12,
 80,
 20,
 98,
 80,
 20,
 20,
 0,
 80,
 98,
 12,
 20,
 20,
 80,
 20,
 20,
 20,
 0,
 20,
 60,
 80,
 20,
 20,
 96,
 20,
 20,
 95,
 20,
 20,
 4,
 95,
 80,
 40,
 20,
 20,
 98,
 20,
 4,
 100,
 50,
 20,
 20,
 80,
 60,
 98,
 20,
 4,
 20,
 100,
 20,
 80,
 80,
 40,
 20,
 60,
 0,
 20,
 20,
 4,
 4,
 20,
 0,
 4,
 80,
 20,
 95,
 95,
 100,
 20,
 95,
 4,
 80,
 95,
 80,
 80,
 20,
 80,
 20,
 20,
 80,
 0,
 60,
 85,
 20,
 98,
 43,
 98,
 80,
 4,
 95,
 98,
 20,
 100,
 80,
 40,
 4,
 85,
 20,
 4,
 20,
 20,
 80,
 20,
 0,
 20,
 60,
 40,
 90,
 96,
 2,
 4]

In [56]:
print(len(test_data))

220


In [57]:
sum(scores)/len(scores)

43.73853211009175